# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
weather_data_file ="weatherdata.csv"
weather_df = pd.read_csv(weather_data_file)
weather_df = weather_df.drop(columns=['Unnamed: 0'])
weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,69.06,90,96,2.26,US,1631499160
1,Ribeira Grande,38.5167,-28.7000,69.69,74,98,20.29,PT,1631499160
2,Hong Kong,22.2855,114.1577,91.36,75,5,1.01,HK,1631499097
3,Cape Town,-33.9258,18.4232,50.88,89,85,1.99,ZA,1631499161
4,Atuona,-9.8000,-139.0333,78.03,68,1,20.85,PF,1631499161


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(api_key=g_key)


In [14]:
locations = weather_df[["Lat", "Lng"]].astype(float)
weight = weather_df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=max(weight),
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
clean_weather_df = weather_df.dropna()
new_weather_df= clean_weather_df.loc[(clean_weather_df["Max Temp"] < 80) & (clean_weather_df["Max Temp"] > 70)]
new_weather_df =clean_weather_df[clean_weather_df["Wind Speed"] < 10]
new_weather_df =clean_weather_df[clean_weather_df["Cloudiness"] == 0]
new_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Santa Rosalía,27.3167,-112.2833,87.13,53,0,2.93,MX,1631499163
23,Rikitea,-23.1203,-134.9692,73.18,73,0,3.11,PF,1631499168
31,Carnarvon,-24.8667,113.6333,68.52,35,0,13.91,AU,1631499171
32,Birjand,32.8663,59.2211,59.83,22,0,6.85,IR,1631499171
33,Viamão,-30.0811,-51.0233,68.74,84,0,9.22,BR,1631499172


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
new_weather_df.loc[:,["City","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Country","Date","Hotel Name"]]
#new_weather_df["Hotel Name"] = ""
hotel_df = new_weather_df
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,Santa Rosalía,27.3167,-112.2833,87.13,53,0,2.93,MX,1631499163,
23,Rikitea,-23.1203,-134.9692,73.18,73,0,3.11,PF,1631499168,
31,Carnarvon,-24.8667,113.6333,68.52,35,0,13.91,AU,1631499171,
32,Birjand,32.8663,59.2211,59.83,22,0,6.85,IR,1631499171,
33,Viamão,-30.0811,-51.0233,68.74,84,0,9.22,BR,1631499172,


In [35]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

target_radius = 5000
target_type = "hotel"

params = {"radius":target_radius, "type": target_type, "key": gkey}

for index, row in hotel_df.iterrows():

    latitude = row["Lat"]
    longitude = row["Lng"]
    params["location"] = f"{latitude},{longitude}"
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
print(json.dumps(response, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "next_page_token": "Aap_uEAQHdzL-RKC9uN9VmK0i9Sz9M268SfLSpXdjLHUsCGlntC3DaC95HINRCeZ4d0GtRL5WyQD0H0IScU9M7LZAO3CG4Fo7J6XgfFNwCXeBTYeiI9I7Tmc8giik0rcp3UgWoqGG60hnqqhhuTYGlD0u00Ip1n5Qeek1zveHbI-pb6RlCdRX2xomhG37ofLfj6ckuODNFzCTJ26xozqkVCSGQaSYCxNRMs9qooPrwkgsWxW7jKSJrZTp7HmIxz_fzCIEqqTCgsyW-XhUYBx3ECQJ0W8t0kCvlxA1Or7Ngsrk1ajhQBM8I7DBcvhKF-ORZ1Dui3TorUlz4ARhJ2EOrkp3QUBd2-59Jo_xbCk8vmtHSln5IJHScB6XdcI0W8PosuYbtQ4nUo20J2AGgKR83ZyS2vq3VBtBiB59u3ucWzbkpJ6i3H8KEqfLQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -38.5544968,
                    "lng": -58.73960879999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -38.51463316917173,
                        "lng": -58.693399519207
                    },
                    "southwest": {
                        "lat": -38.60644906676189,
                        "lng": -58.8191

In [36]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,Santa Rosalía,27.3167,-112.2833,87.13,53,0,2.93,MX,1631499163,Santa Rosalía
23,Rikitea,-23.1203,-134.9692,73.18,73,0,3.11,PF,1631499168,Rikitea
31,Carnarvon,-24.8667,113.6333,68.52,35,0,13.91,AU,1631499171,Carnarvon
32,Birjand,32.8663,59.2211,59.83,22,0,6.85,IR,1631499171,Birjand
33,Viamão,-30.0811,-51.0233,68.74,84,0,9.22,BR,1631499172,Viamão


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))